# 03 — Batch Map Processing

Generate high-resolution maps for **all** control grid cells and save them to the Google Drive output directory.

**What this notebook covers:**
- Loading config and all boundary data
- Iterating over grid cells with a progress bar
- Saving maps to `02_outputs/generated_maps/` on Google Drive
- Summary of generated files

**Tip:** For fully automated runs, use `python scripts/generate_all_maps.py` instead.

## Setup

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.config_loader import load_config, get_data_dir, get_output_dir
from src.data_processing.load_boundaries import (
    load_control_grid,
    load_layer,
    validate_crs,
)
from src.mapping.map_generator import MapGenerator

config = load_config()
data_dir = get_data_dir(config)
output_dir = get_output_dir(config)

print(f"Data dir:   {data_dir}")
print(f"Output dir: {output_dir}")

## Load All Data

In [ ]:
grid = load_control_grid(data_dir)
grid = validate_crs(grid)

roads = load_layer(data_dir, "roads")
buildings = load_layer(data_dir, "buildings")

print(f"\nTotal grid cells to process: {len(grid)}")
print(f"Columns: {list(grid.columns)}")

## Configure Grid ID Column

Set `grid_id_col` to the column containing unique cell identifiers. If `None`, row index will be used.

In [ ]:
# Set this to match your data (e.g. "CELL_ID", "grid_id", "ID")
grid_id_col = None

# Auto-detect from common names
if grid_id_col is None:
    for candidate in ["CELL_ID", "cell_id", "grid_id", "GRID_ID", "ID", "id", "NAME", "name"]:
        if candidate in grid.columns:
            grid_id_col = candidate
            print(f"Auto-detected grid ID column: {grid_id_col}")
            break

if grid_id_col:
    print(f"Using column: {grid_id_col}")
    print(f"Sample IDs: {grid[grid_id_col].head().tolist()}")
else:
    print("No ID column found — will use zero-padded row index (001, 002, ...)")

## Initialize Map Generator

Maps are saved to `<output_dir>/generated_maps/grid_cell_<ID>/grid_cell_<ID>_map.png`.

In [ ]:
map_settings = config.get("map_settings", {})

generator = MapGenerator(
    output_dir=output_dir / "generated_maps",
    fig_width=map_settings.get("fig_width", 19.2),
    fig_height=map_settings.get("fig_height", 10.8),
    dpi=map_settings.get("dpi", 100),
    add_basemap=map_settings.get("add_basemap", True),
)

print(f"Output directory: {generator.output_dir}")
print(f"Resolution:       {generator.fig_width * generator.dpi:.0f} x {generator.fig_height * generator.dpi:.0f} px")
print(f"Basemap:          {generator.add_basemap}")

## Generate All Maps

This iterates over every grid cell and generates a PNG map. Progress is tracked with `tqdm`.

In [ ]:
from tqdm.notebook import tqdm

generated_paths = []
errors = []

for idx, (_, row) in enumerate(tqdm(grid.iterrows(), total=len(grid), desc="Generating maps")):
    # Determine grid ID
    if grid_id_col:
        grid_id = str(row[grid_id_col])
    else:
        grid_id = str(idx + 1).zfill(3)

    cell = grid.iloc[[idx]]

    try:
        path = generator.generate_map(
            grid_cell=cell,
            grid_id=grid_id,
            all_grid_cells=grid,
            roads=roads,
            buildings=buildings,
        )
        generated_paths.append(path)
    except Exception as e:
        errors.append((grid_id, str(e)))
        print(f"Error on grid cell {grid_id}: {e}")

print(f"\nGenerated: {len(generated_paths)} maps")
if errors:
    print(f"Errors:    {len(errors)}")
    for gid, err in errors:
        print(f"  - {gid}: {err}")

## Summary

List the generated files and their sizes.

In [ ]:
import pandas as pd

if generated_paths:
    summary = pd.DataFrame({
        "file": [p.name for p in generated_paths],
        "size_kb": [p.stat().st_size / 1024 for p in generated_paths],
        "path": [str(p) for p in generated_paths],
    })

    print(f"Total maps:       {len(summary)}")
    print(f"Total size:       {summary['size_kb'].sum() / 1024:.1f} MB")
    print(f"Avg size per map: {summary['size_kb'].mean():.0f} KB")
    print(f"\nOutput folder: {generator.output_dir}")

    summary.head(10)
else:
    print("No maps generated.")

## View a Sample Output

Display the first generated map to spot-check quality.

In [ ]:
from IPython.display import Image, display

if generated_paths:
    print(f"Showing: {generated_paths[0].name}")
    display(Image(filename=str(generated_paths[0]), width=960))